In [1]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2 contractions unidecode
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('./spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
from pyspark.ml import PipelineModel, Pipeline
import PyPDF2
import os

In [3]:
spark = SparkSession.builder \
    .appName("nlpdemo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp_2.12:6.0.2"
            ])) \
    .getOrCreate()

25/07/27 16:09:13 WARN Utils: Your hostname, Joses-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.105.113.167 instead (on interface en0)
25/07/27 16:09:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/joseaguilar/.ivy2/cache
The jars for the packages stored in: /Users/joseaguilar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-093281f0-4374-44d5-bdf7-45a2225213a2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.2 in central


:: loading settings :: url = jar:file:/Users/joseaguilar/Documents/Development/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in local-m2-cache
	found org.apache.httpcomponents#httpclient;4.5.13 in local-m2-cache
	found org.apache.httpcomponents#httpcore;4.4.13 in local-m2-cache
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1

In [4]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start()

6.0.2


25/07/27 16:09:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [22]:
palabras_ejemplo = [
    ("computation",), ("computer",), ("computing",),
    ("running",), ("ran",), ("run",),
    ("playing",), ("played",), ("play",),
    ("better",), ("best",), ("good",),
    ("studies",), ("study",), ("studying",),
    ("wolves",), ("wolf",), ("wolfing",),
]
df_palabras = spark.createDataFrame(palabras_ejemplo, ["palabra"])
df_palabras.show()

+-----------+
|    palabra|
+-----------+
|computation|
|   computer|
|  computing|
|    running|
|        ran|
|        run|
|    playing|
|     played|
|       play|
|     better|
|       best|
|       good|
|    studies|
|      study|
|   studying|
|     wolves|
|       wolf|
|    wolfing|
+-----------+



In [23]:
from sparknlp.annotator import Stemmer

# Preparar DataFrame con la palabra como documento (Spark NLP espera texto en DocumentAssembler)
df_palabras = df_palabras.withColumnRenamed("palabra", "texto")
stem_assembler = DocumentAssembler().setInputCol("texto").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
stemmer = Stemmer().setInputCols(["token"]).setOutputCol("raiz")
stem_finisher = Finisher().setInputCols(["raiz"]).setOutputCols(["palabra_stem"])

pipeline_stem = Pipeline(stages=[stem_assembler, tokenizer,stemmer, stem_finisher])
df_stemmed = pipeline_stem.fit(df_palabras).transform(df_palabras)
df_stemmed.select("texto", "palabra_stem").show()


+-----------+------------+
|      texto|palabra_stem|
+-----------+------------+
|computation|    [comput]|
|   computer|    [comput]|
|  computing|    [comput]|
|    running|       [run]|
|        ran|       [ran]|
|        run|       [run]|
|    playing|     [playe]|
|     played|     [playe]|
|       play|      [plai]|
|     better|    [better]|
|       best|      [best]|
|       good|      [good]|
|    studies|     [studi]|
|      study|     [studi]|
|   studying|     [studi]|
|     wolves|      [wolv]|
|       wolf|      [wolf]|
|    wolfing|      [wolf]|
+-----------+------------+



In [26]:
from sparknlp.annotator import LemmatizerModel

# Cargar modelo pre-entrenado de lematización en inglés
lemmatizer = LemmatizerModel.pretrained("lemma_antbnc", "en")\
    .setInputCols(["token"]).setOutputCol("lemma")
# Nota: 'lemma_antbnc' es un modelo de lematización inglés. Para español se requeriría un diccionario adicional.
df_nuevas_palabras = df_stemmed.select("texto", "palabra_stem")
# Necesitamos tokenizar las palabras para alimentar el Lemmatizer
token_assembler = DocumentAssembler().setInputCol("texto").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
pipeline_lemma = Pipeline(stages=[token_assembler, tokenizer, lemmatizer, Finisher().setInputCols(["lemma"]).setOutputCols(["lematizado"])])
df_lemmas = pipeline_lemma.fit(df_nuevas_palabras).transform(df_nuevas_palabras)
df_lemmas.select("texto", "palabra_stem","lematizado").show()


lemma_antbnc download started this may take some time.


25/07/27 16:31:20 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


Approximate size to download 907.6 KB
[OK!]
+-----------+------------+-------------+
|      texto|palabra_stem|   lematizado|
+-----------+------------+-------------+
|computation|    [comput]|[computation]|
|   computer|    [comput]|   [computer]|
|  computing|    [comput]|  [computing]|
|    running|       [run]|        [run]|
|        ran|       [ran]|        [run]|
|        run|       [run]|        [run]|
|    playing|     [playe]|       [play]|
|     played|     [playe]|       [play]|
|       play|      [plai]|       [play]|
|     better|    [better]|       [well]|
|       best|      [best]|       [good]|
|       good|      [good]|       [good]|
|    studies|     [studi]|      [study]|
|      study|     [studi]|      [study]|
|   studying|     [studi]|      [study]|
|     wolves|      [wolv]|       [wolf]|
|       wolf|      [wolf]|       [wolf]|
|    wolfing|      [wolf]|       [wolf]|
+-----------+------------+-------------+



In [9]:
from pyspark.ml.feature import NGram

from sparknlp.annotator import Tokenizer, StopWordsCleaner

tokenizer = Tokenizer().setInputCols(["document_norm"]).setOutputCol("token")
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols(["token"]).setOutputCol("token_sin_stop")\
    .setCaseSensitive(False)\
    .setStopWords(["y", "de", "la", "the", "and", "is", "es", "porque"])  # lista combinada EN/ES como ejemplo

pipeline_stop = Pipeline(stages=[DocumentAssembler().setInputCol("texto").setOutputCol("document_norm"),
                                 tokenizer, stopwords_cleaner,
                                 Finisher().setInputCols(["token_sin_stop"]).setOutputCols(["tokens_final"])])

# Crear algunos textos de ejemplo para n-gramas
oraciones = [("to be or not to be",), ("all your base are belong to us",)]
df_oraciones = spark.createDataFrame(oraciones, ["texto"])
# Tokenizar primero (utilizando la pipeline previa de tokenización simple)
df_tokens = pipeline_stop.fit(df_oraciones).transform(df_oraciones)  # reutilizamos pipeline_stop de antes para tokenizar sin stopwords
# Generar bigramas
ngrammer = NGram(n=2, inputCol="tokens_final", outputCol="bigrams")
df_bigrams = ngrammer.transform(df_tokens)
df_bigrams.select("tokens_final", "bigrams").show(truncate=False)


+--------------------------------------+-------------------------------------------------------------+
|tokens_final                          |bigrams                                                      |
+--------------------------------------+-------------------------------------------------------------+
|[to, be, or, not, to, be]             |[to be, be or, or not, not to, to be]                        |
|[all, your, base, are, belong, to, us]|[all your, your base, base are, are belong, belong to, to us]|
+--------------------------------------+-------------------------------------------------------------+

